<a href="https://colab.research.google.com/github/Camilo-a17/Camilo-a17/blob/main/Analisis_threads_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly.offline import iplot
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX


#Importamos el dataset
df = pd.read_csv("threads_reviews.csv")

In [35]:
print(df.columns)
df.describe()

Index(['source', 'review_description', 'rating', 'review_date'], dtype='object')


,rating
count,40435.000000
mean,3.300532
std,1.773157
min,1.000000
25%,1.000000
50%,4.000000
75%,5.000000
max,5.000000


In [36]:
df.head()

,source,review_description,rating,review_date
0,Google Play,Very good app for Android phone and me,5,27-08-2023 10:31
1,Google Play,Sl👍👍👍👍,5,27-08-2023 10:28
2,Google Play,Best app,5,27-08-2023 9:47
3,Google Play,Gatiya app,1,27-08-2023 9:13
4,Google Play,Lit bruv,5,27-08-2023 9:00


In [37]:
colors = ['#494BD3', '#E28AE2', '#F1F481', '#79DB80', '#DF5F5F',
          '#69DADE', '#C2E37D', '#E26580', '#D39F49', '#B96FE3']

def plot_top(df, column, title, x_label, y_label, top, template='plotly_dark'):
    pl = df[column].value_counts().head(top)
    fig = px.bar(y=pl.values,
                 x=pl.index,
                 color_discrete_sequence=px.colors.sequential.PuBuGn,
                 text=pl.values,
                 title=title,
                 template=template)
    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label,
        font=dict(size=10, family="Franklin Gothic"))
    fig.show()

def cat_summary_with_graph(dataframe, col_name):
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('Countplot', 'Percentages'),
                        specs=[[{"type": "xy"}, {'type': 'domain'}]])

    fig.add_trace(go.Bar(y=dataframe[col_name].value_counts().values.tolist(),
                         x=[str(i) for i in dataframe[col_name].value_counts().index],
                         text=dataframe[col_name].value_counts().values.tolist(),
                         textfont=dict(size=15),
                         name=col_name,
                         textposition='auto',
                         showlegend=False,
                         marker=dict(color=colors,
                                     line=dict(color='#DBE6EC',
                                               width=1))),
                  row=1, col=1)

    fig.add_trace(go.Pie(labels=dataframe[col_name].value_counts().keys(),
                         values=dataframe[col_name].value_counts().values,
                         textfont=dict(size=20),
                         textposition='auto',
                         showlegend=False,
                         name=col_name,
                         marker=dict(colors=colors)),
                  row=1, col=2)

    fig.update_layout(title={'text': col_name,
                             'y': 0.9,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')

    iplot(fig)


In [38]:
# Para crear un gráfico de barras
plot_top(df, 'rating', 'Distribución de Calificaciones', 'Calificación', 'Conteo', 5)

# Para crear un gráfico de pastel y un gráfico de barras en un subplot
cat_summary_with_graph(df, 'source')


In [39]:
# Para crear un gráfico de barras
plot_top(df, 'rating', 'Distribución de Calificaciones', 'Calificación', 'Conteo', 5)

# Para crear un gráfico de pastel y un gráfico de barras en un subplot
cat_summary_with_graph(df, 'rating')


In [40]:
df['review_date'] = pd.to_datetime(df['review_date'])


In [41]:
min_date = df['review_date'].min()


In [42]:
from dateutil.relativedelta import relativedelta

# Lista para almacenar las fechas de inicio de cada mes
start_dates = []

# Fecha inicial
current_date = min_date

# Generar las fechas de inicio de los 10 meses
for _ in range(10):
    start_dates.append(current_date)
    current_date += relativedelta(months=1)


In [43]:
monthly_ratings = []

for i in range(len(start_dates) - 1):
    start_date = start_dates[i]
    end_date = start_dates[i + 1]

    # Filtrar las calificaciones dentro del período
    ratings_within_period = df[(df['review_date'] >= start_date) & (df['review_date'] < end_date)]

    # Contar el número de calificaciones en el período
    num_ratings = len(ratings_within_period)

    monthly_ratings.append(num_ratings)

# Agregar el último mes
end_date = start_dates[-1] + relativedelta(months=1)
ratings_within_period = df[(df['review_date'] >= start_dates[-1]) & (df['review_date'] < end_date)]
num_ratings = len(ratings_within_period)
monthly_ratings.append(num_ratings)


In [44]:
# Crear un DataFrame con las fechas y el número de calificaciones
monthly_ratings_df = pd.DataFrame({'Start Date': start_dates, 'Number of Ratings': monthly_ratings})

# Crear el gráfico de barras
fig = px.bar(monthly_ratings_df, x='Start Date', y='Number of Ratings',
             labels={'Number of Ratings': 'Número de Calificaciones'},
             title='Número de Calificaciones por Mes',
             color_discrete_sequence=['#494BD3'])

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Fecha de Inicio del Mes',
                  yaxis_title='Número de Calificaciones',
                  xaxis_tickangle=-45)

# Mostrar el gráfico
fig.show()


In [45]:
df['review_date'] = pd.to_datetime(df['review_date'])
df.set_index('review_date', inplace=True)

# Resample los datos a nivel mensual y cuenta el número de calificaciones por mes
monthly_ratings = df['rating'].resample('M').count()

In [46]:
pip install pmdarima


In [47]:
# Ajustar manualmente los hiperparámetros del modelo ARIMA
p = 1  # Orden AR
d = 1  # Orden de diferenciación
q = 1  # Orden MA
P = 1  # Orden AR estacional
D = 1  # Orden de diferenciación estacional
Q = 1  # Orden MA estacional
s = 12  # Período estacional

# Crear y entrenar el modelo SARIMA con los hiperparámetros especificados
sarima_model = SARIMAX(monthly_ratings, order=(p, d, q), seasonal_order=(P, D, Q, s))
sarima_results = sarima_model.fit()

# Realizar una predicción para los próximos 10 meses
forecast_steps = 10
forecast = sarima_results.get_forecast(steps=forecast_steps)

# Obtener las predicciones y los intervalos de confianza
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.



In [48]:
# Crear una figura de Plotly
fig = go.Figure()

# Agregar la serie temporal observada
fig.add_trace(go.Scatter(x=monthly_ratings.index, y=monthly_ratings, mode='lines', name='Observado'))

# Agregar las predicciones y el intervalo de confianza
fig.add_trace(go.Scatter(x=forecast_mean.index, y=forecast_mean, mode='lines', name='Predicción', line=dict(color='red')))
fig.add_trace(go.Scatter(x=forecast_ci.index, y=forecast_ci.iloc[:, 0], fill=None, mode='lines', line=dict(color='pink'), name='Intervalo de Confianza'))
fig.add_trace(go.Scatter(x=forecast_ci.index, y=forecast_ci.iloc[:, 1], fill='tonexty', mode='lines', line=dict(color='pink'), name='Intervalo de Confianza'))

# Personalizar el diseño del gráfico
fig.update_layout(title='Predicción de Usuarios Activos para los Próximos 10 Meses (ARIMA)',
                  xaxis_title='Fecha',
                  yaxis_title='Número de Calificaciones',
                  legend=dict(x=0, y=1),
                  showlegend=True,
                  hovermode='x')

# Mostrar el gráfico
fig.show()


#Analisis de sentimientos

In [49]:
!pip install nltk
!pip install textblob


In [50]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [51]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Función para realizar análisis de sentimiento con TextBlob
def analyze_sentiment_textblob(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'Positivo'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negativo'

# Función para realizar análisis de sentimiento con NLTK Vader
def analyze_sentiment_vader(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    if sentiment['compound'] >= 0.05:
        return 'Positivo'
    elif sentiment['compound'] <= -0.05:
        return 'Negativo'
    else:
        return 'Neutral'


In [53]:
# Aplicar el análisis de sentimiento a tu DataFrame (reemplaza 'columna_comentarios' con el nombre real de tu columna)
df['sentimiento_textblob'] = df['review_description'].apply(analyze_sentiment_textblob)
df['sentimiento_vader'] = df['review_description'].apply(analyze_sentiment_vader)


In [55]:
# Análisis de sentimiento con TextBlob
sentimiento_textblob_counts = df['sentimiento_textblob'].value_counts().reset_index()
sentimiento_textblob_counts.columns = ['Sentimiento', 'Cantidad']

# Gráfico de barras interactivo con Plotly
fig = px.bar(sentimiento_textblob_counts, x='Sentimiento', y='Cantidad', color='Sentimiento',
             labels={'Sentimiento': 'Sentimiento', 'Cantidad': 'Cantidad'},
             title='Análisis de Sentimiento (TextBlob)')

# Personaliza el diseño del gráfico
fig.update_layout(xaxis_title='Sentimiento', yaxis_title='Cantidad', showlegend=True)

# Muestra el gráfico
fig.show()

# Análisis de sentimiento con NLTK Vader
sentimiento_vader_counts = df['sentimiento_vader'].value_counts().reset_index()
sentimiento_vader_counts.columns = ['Sentimiento', 'Cantidad']

# Gráfico de barras interactivo con Plotly
fig = px.bar(sentimiento_vader_counts, x='Sentimiento', y='Cantidad', color='Sentimiento',
             labels={'Sentimiento': 'Sentimiento', 'Cantidad': 'Cantidad'},
             title='Análisis de Sentimiento (NLTK Vader)')

# Personaliza el diseño del gráfico
fig.update_layout(xaxis_title='Sentimiento', yaxis_title='Cantidad', showlegend=True)

# Muestra el gráfico
fig.show()
